## Autonomous Multi-Agent System for Enhanced Automotive User Experience

#### Capstone Project - Kaggle 5-Day Generative AI Course

In [1]:
!pip uninstall -qqy kfp jupyterlab google-generativeai tensorflow-hub 2>/dev/null
!pip install -U -q langchain langchain-google-genai langgraph google-cloud-aiplatform pandas scikit-learn "langchain-community>=0.2.7" "langchain-core>=0.2.19"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 7.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take i

In [2]:
import os
import typing
import uuid
from collections.abc import Iterable
from random import randint
from pprint import pprint

from langchain_core.messages.ai import AIMessage
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.tool import ToolMessage
from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from google.cloud import aiplatform
from langchain_community.embeddings import VertexAIEmbeddings
from langchain_community.vectorstores.matching_engine import MatchingEngine

from IPython.display import Image, display, Markdown

from typing import Annotated, Literal
from typing_extensions import TypedDict

In [3]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [4]:
PROJECT_ID = "capstone-project"
LOCATION = "us-central1"

try:
    aiplatform.init(project=PROJECT_ID, location=LOCATION)
    print(f"Vertex AI initialized for project {PROJECT_ID} in {LOCATION}.")
except Exception as e:
    print(f"Error initializing Vertex AI: {e}")
    print("Please ensure you have entered a valid PROJECT_ID and have the necessary permissions.")


Vertex AI initialized for project capstone-project in us-central1.


In [5]:
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]
    current_request: str | None
    last_agent_response: str | None
    route_decision: Literal["Navigation", "Media", "Communication", "VehicleInfo", "Respond", "Error", None]
    vehicle_context: list[str] | None
    navigation_context: dict | None
    media_context: dict | None
    communication_context: dict | None
    error: str | None
    final_response_generated: bool

## Tools

In [6]:
@tool
def navigation_tool(action: str, destination: str | None = None, poi_type: str | None = None) -> str:
    """
    Simulates interaction with a Navigation API.
    Actions can be 'find_poi', 'start_navigation', 'get_traffic'.
    Requires destination for 'start_navigation'.
    Requires poi_type for 'find_poi' (e.g., 'restaurant', 'gas station').
    """
    print(f"--- TOOL CALL: navigation_tool (Action: {action}, Destination: {destination}, POI: {poi_type}) ---")
    if action == "find_poi":
        return f"Simulated: Found nearest {poi_type or 'POI'}."
    elif action == "start_navigation" and destination:
        return f"Simulated: Starting navigation to {destination}."
    elif action == "get_traffic":
        return "Simulated: Traffic is moderate."
    else:
        return "Simulated: Invalid navigation action or missing parameters."

@tool
def media_tool(action: str, query: str | None = None) -> str:
    """
    Simulates interaction with a Media API.
    Actions can be 'play', 'pause', 'search_music', 'search_podcast'.
    Requires query for search actions.
    """
    print(f"--- TOOL CALL: media_tool (Action: {action}, Query: {query}) ---")
    if action == "play" and query:
        return f"Simulated: Playing '{query}'."
    elif action == "play":
        return f"Simulated: Resuming playback."
    elif action == "pause":
        return "Simulated: Pausing media."
    elif action == "search_music" and query:
        return f"Simulated: Found music matching '{query}'."
    elif action == "search_podcast" and query:
         return f"Simulated: Found podcasts matching '{query}'."
    else:
        return "Simulated: Invalid media action or missing query."

@tool
def communication_tool(action: str, recipient: str | None = None, message: str | None = None) -> str:
    """
    Simulates interaction with a Communication API (e.g., SMS, Email).
    Actions can be 'send_message', 'read_last_message'.
    Requires recipient and message for 'send_message'.
    """
    print(f"--- TOOL CALL: communication_tool (Action: {action}, Recipient: {recipient}) ---")
    if action == "send_message" and recipient and message:
        return f"Simulated: Sending message '{message}' to {recipient}."
    elif action == "read_last_message":
        return "Simulated: Last message reads 'On my way!'"
    else:
        return "Simulated: Invalid communication action or missing parameters."

@tool
def get_vehicle_info(query: str) -> str:
    """
    Provides information about the vehicle's features and functions by
    searching the car manual. Use this for questions about how to operate
    the car or troubleshoot issues.
    """
    print(f"--- TOOL CALL: get_vehicle_info (Query: {query}) ---")
    if "headlight" in query.lower():
        return "Simulated RAG: To turn on the headlights, rotate the dial on the left stalk clockwise."
    elif "tire pressure" in query.lower():
         return "Simulated RAG: Recommended tire pressure is 35 PSI for all tires."
    else:
        return "Simulated RAG: Information not found in the manual for that query."

available_tools = [navigation_tool, media_tool, communication_tool, get_vehicle_info]